<a href="https://colab.research.google.com/github/yt203y/20240522_GROMACS_introduction-fromepcced/blob/gh-pages/2023_workshop_Torun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bootstrap

Prepare necessary files:

(you can try "Edit" > "Clear all outputs" after it's been done)

In [ ]:
!pip install ase
!pip install torchani
!pip install pyscf
!pip install mkl-service

In [ ]:
!wget http://mlatom.com/wp-content/uploads/2023/12/2023_workshop_Torun.zip -O 2023_workshop_Torun.zip
!unzip 2023_workshop_Torun.zip

# Install and use MLatom

Install MLatom via PyPI

In [ ]:
!pip install mlatom

To use KREG_API in Colab:

In [ ]:
!wget http://mlatom.com/wp-content/uploads/2023/12/fortran.zip
!unzip fortran.zip -d /usr/local/lib/python3.10/dist-packages/mlatom/

Run MLatom in command line:

In [ ]:
!mlatom

Run MLatom in Python:

In [ ]:
import mlatom as ml

# Run this tutorial on XACS cloud

You can also run this tutorial on XACS cloud (https://xacs.xmu.edu.cn/newcloud/), where some extra programs are already install to enable features like AIQM1 method.

Necessary files can be found in ``/export/home/xacscloud/workshop2023/2023_workshop_Torun/`` once you enter the terminal on XACS cloud.

You can copy the folder to your directory:

``cp -r /export/home/xacscloud/workshop2023/2023_workshop_Torun/ .``

Then set up the environment:

``cd 2023_workshop_Torun && . env.sh``

You can submit command-line jobs with:

``xacs submit xxx.inp``

or

``srun -c4 mlatom xxx.inp``

Python jobs with:

``srun -c4 xxx.py``

or run Python interactively:

``srun -c4 --pty python``



# Train and use models for hydrogen molecule

## KREG

In [ ]:
!cat h2_train_KREG.inp

In [ ]:
!mlatom h2_train_KREG.inp

In [ ]:
import mlatom as ml

# load data set
molDB = ml.data.molecular_database.from_xyz_file('h2.xyz')
molDB.add_scalar_properties_from_file('E_FCI_451.dat', 'energy')

# define the model
model = ml.models.kreg(model_file='energies')

# split data set for optimizing hyperparameters
[subtraining_molDB, validation_molDB] = ml.data.sample(number_of_splits=2, fraction_of_points_in_splits=[0.8, 0.2], molecular_database_to_split=molDB, sampling='random')

# optimize hyperparameters
model.hyperparameters["sigma"].minval = 2**-4
model.optimize_hyperparameters(subtraining_molecular_database=subtraining_molDB,
                                     validation_molecular_database=validation_molDB,
                                     optimization_algorithm='nelder-mead',
                                     hyperparameters=['lambda', 'sigma'],
                                     training_kwargs={'property_to_learn': 'energy'},
                                     prediction_kwargs=None)
lmbd = model.hyperparameters['lambda'].value
sigma = model.hyperparameters['sigma'].value
print(f'Optimized hyperparameters: lambda={lmbd}, sigma={sigma}')

# train the final model
model.train(molecular_database=molDB, property_to_learn='energy')

In [ ]:
!cat h2_opt_KREG.inp
!echo -------------------------------------------------------------------
!cat h2_init.xyz

In [ ]:
!mlatom h2_opt_KREG.inp

In [ ]:
!cat eq_KREG.xyz

In [ ]:
import mlatom as ml

# load initial geometry
mol = ml.data.molecule.from_xyz_file('h2_init.xyz')
print(mol.get_xyz_string())

# load the model
model = ml.models.kreg(model_file='energies')

# run geometry optimization
ml.optimize_geometry(model=model, molecule=mol, program='ASE')
print(mol.get_xyz_string())

## TorchANI

In [ ]:
!cat h2_train_ANI.inp

In [ ]:
!mlatom h2_train_ANI.inp

In [ ]:
import mlatom as ml

# load data set
molDB = ml.data.molecular_database.from_xyz_file('h2.xyz')
molDB.add_scalar_properties_from_file('E_FCI_451.dat', 'energy')

# define the model
model = ml.models.ani(model_file='energies_ani_api.pt', hyperparameters={'max_epochs': 16})

# train the final model
model.train(molecular_database=molDB, property_to_learn='energy')

In [ ]:
!cat h2_opt_ANI.inp

In [ ]:
!mlatom h2_opt_ANI.inp

In [ ]:
!cat eq_ANI.xyz

In [ ]:
import mlatom as ml

# load initial geometry
mol = ml.data.molecule.from_xyz_file('h2_init.xyz')
print(mol.get_xyz_string())

# load the model
model = ml.models.ani(model_file='energies_ani_api.pt')

# run geometry optimization
ml.optimize_geometry(model=model, molecule=mol, program='ASE')
print(mol.get_xyz_string())

# Estimate model accuracy

## KREG

In [ ]:
!cat ethanol_estAcc_KREG.inp

In [ ]:
!mlatom ethanol_estAcc_KREG.inp

## TorchANI

In [ ]:
!cat ethanol_estAcc_ANI.inp

In [ ]:
!mlatom ethanol_estAcc_ANI.inp # this training will take a lot of time

# Methods

In [ ]:
!cat h2_vinylacetylene_init.xyz

## ANI-1ccx

In [ ]:
!cat h2_vinylacetylene_opt_ANI1ccx.inp

In [ ]:
!mlatom h2_vinylacetylene_opt_ANI1ccx.inp

In [ ]:
import mlatom as ml
from mlatom import constants
# load initial geometries
molDB = ml.data.molecular_database.from_xyz_file('h2_vinylacetylene_init.xyz')
# define the model with methods
model = ml.models.methods(method='ANI-1ccx')
for mol in molDB:
    print(mol.get_xyz_string())
    # run geometry optimization
    ml.optimize_geometry(model=model, molecule=mol, program='ASE')
    print(mol.get_xyz_string())

In [ ]:
!cat h2_vinylacetylene_freq_ANI1ccx.inp

In [ ]:
!mlatom h2_vinylacetylene_freq_ANI1ccx.inp

In [ ]:
import mlatom as ml
from mlatom import constants
# load initial geometries
molDB = ml.data.molecular_database.from_xyz_file('final_ANI1ccx.xyz')
molDB[0].shape = 'linear'
# define the model with methods
model = ml.models.methods(method='ANI-1ccx')
for mol in molDB:
    ml.thermochemistry(model=model, molecule=mol, program='ASE')
    fmt = ' %-41s: %15.5f Hartree'
    print(fmt % ('Standard deviation of NNs', mol.ani1ccx.energy_standard_deviation), end='')
    print(' %15.5f kcal/mol' % (mol.ani1ccx.energy_standard_deviation * constants.Hartree2kcalpermol))
    print(fmt % ('ZPE-exclusive internal energy at      0 K', mol.energy))
    print(fmt % ('Zero-point vibrational energy', mol.ZPE))
    print(fmt % ('Internal energy                  at   0 K', mol.U0))
    print(fmt % ('Enthalpy                         at 298 K', mol.H))
    print(fmt % ('Gibbs free energy                at 298 K', mol.G))
    if 'DeltaHf298' in mol.__dict__:
        print('')
        fmt = ' %-41s: %15.5f Hartree %15.5f kcal/mol'
        print(fmt % ('Atomization enthalpy             at   0 K', mol.atomization_energy_0K, mol.atomization_energy_0K * constants.Hartree2kcalpermol))
        print(fmt % ('ZPE-exclusive atomization energy at   0 K', mol.ZPE_exclusive_atomization_energy_0K, mol.ZPE_exclusive_atomization_energy_0K * constants.Hartree2kcalpermol))
        print(fmt % ('Heat of formation                at 298 K', mol.DeltaHf298, mol.DeltaHf298 * constants.Hartree2kcalpermol))
        if mol.ani1ccx.energy_standard_deviation > 1.68*constants.kcalpermol2Hartree:
            print(' * Warning * Heat of formation have high uncertainty!')

## AIQM1

To use AIQM1, extra programs are required. Please move from Colab to XACS cloud (https://xacs.xmu.edu.cn/newcloud/).
Necessary files can be found in /export/home/xacscloud/workshop2023/2023_workshop_Torun/

In [ ]:
!cat h2_vinylacetylene_opt_AIQM1.inp

In [ ]:
!mlatom h2_vinylacetylene_opt_AIQM1.inp

In [ ]:
import mlatom as ml
from mlatom import constants
# load initial geometries
molDB = ml.data.molecular_database.from_xyz_file('h2_vinylacetylene_init.xyz')
molDB[0].shape = 'linear'
# define the model with methods
model = ml.models.methods(method='AIQM1')
for mol in molDB:
    print(mol.get_xyz_string())
    # run geometry optimization
    ml.optimize_geometry(model=model, molecule=mol, program='ASE')
    print(mol.get_xyz_string())

In [ ]:
!cat h2_vinylacetylene_freq_AIQM1.inp

In [ ]:
!mlatom h2_vinylacetylene_freq_AIQM1.inp

In [ ]:
import mlatom as ml
from mlatom import constants
# load initial geometries
molDB = ml.data.molecular_database.from_xyz_file('final_AIQM1.xyz')
molDB[0].shape = 'linear'
# define the model with methods
model = ml.models.methods(method='AIQM1')
for mol in molDB:
    # run thermochemistry calculations
    ml.thermochemistry(model=model, molecule=mol, program='ASE')
    fmt = ' %-41s: %15.5f Hartree'
    print(fmt % ('Standard deviation of NN contribution', mol.aiqm1_nn.energy_standard_deviation), end='')
    print(' %15.5f kcal/mol' % (mol.aiqm1_nn.energy_standard_deviation * constants.Hartree2kcalpermol))
    print(fmt % ('NN contribution', mol.aiqm1_nn.energy))
    print(fmt % ('Sum of atomic self energies', mol.aiqm1_atomic_energy_shift.energy))
    print(fmt % ('ODM2* contribution', mol.odm2star.energy))
    print(fmt % ('D4 contribution', mol.d4_wb97x.energy))
    print(fmt % ('Total energy', mol.energy))
    print(fmt % ('ZPE-exclusive internal energy at      0 K', mol.energy))
    print(fmt % ('Zero-point vibrational energy', mol.ZPE))
    print(fmt % ('Internal energy                  at   0 K', mol.U0))
    print(fmt % ('Enthalpy                         at 298 K', mol.H))
    print(fmt % ('Gibbs free energy                at 298 K', mol.G))
    if 'DeltaHf298' in mol.__dict__:
        print('')
        fmt = ' %-41s: %15.5f Hartree %15.5f kcal/mol'
        print(fmt % ('Atomization enthalpy             at   0 K', mol.atomization_energy_0K, mol.atomization_energy_0K * constants.Hartree2kcalpermol))
        print(fmt % ('ZPE-exclusive atomization energy at   0 K', mol.ZPE_exclusive_atomization_energy_0K, mol.ZPE_exclusive_atomization_energy_0K * constants.Hartree2kcalpermol))
        print(fmt % ('Heat of formation                at 298 K', mol.DeltaHf298, mol.DeltaHf298 * constants.Hartree2kcalpermol))
        if mol.aiqm1_nn.energy_standard_deviation > 0.41*constants.kcalpermol2Hartree:
            print(' * Warning * Heat of formation have high uncertainty!')

## B3LYP

In [ ]:
!cat h2_vinylacetylene_opt_B3LYP.inp

In [ ]:
!mlatom h2_vinylacetylene_opt_B3LYP.inp

In [ ]:
import mlatom as ml
import numpy as np
# load initial geometries
molDB = ml.data.molecular_database.from_xyz_file('h2_vinylacetylene_init.xyz')
molDB[0].shape = 'linear'
# define the model with methods
model = ml.models.methods(method='B3LYP/6-31G*', program='PySCF')
for mol in molDB:
    print(mol.get_xyz_string())
    # run geometry optimization
    ml.optimize_geometry(model=model, molecule=mol, program='ASE')
    print(mol.get_xyz_string())

In [ ]:
!cat h2_vinylacetylene_freq_B3LYP.inp

In [ ]:
!mlatom h2_vinylacetylene_freq_B3LYP.inp

In [ ]:
import mlatom as ml
import numpy as np
# load initial geometries
molDB = ml.data.molecular_database.from_xyz_file('final_B3LYP.xyz')
molDB[0].shape = 'linear'
# define the model with methods
model = ml.models.methods(method='B3LYP/6-31G*', program='PySCF')
for mol in molDB:
    # run thermochemistry calculations
    ml.thermochemistry(model=model, molecule=mol, program='ASE')
    fmt = ' %-41s: %15.5f Hartree'
    print(fmt % ('ZPE-exclusive internal energy at      0 K', mol.energy))
    print(fmt % ('Zero-point vibrational energy', mol.ZPE))
    print(fmt % ('Internal energy                  at   0 K', mol.U0))
    print(fmt % ('Enthalpy                         at 298 K', mol.H))
    print(fmt % ('Gibbs free energy                at 298 K', mol.G))


# Molecular dynamics

※ you need to have a KREG model trained in the previous section before running MD.

In [ ]:
!cat h2_md_kreg.inp

In [ ]:
!cat h2_md_kreg_init.xyz

In [ ]:
!mlatom h2_md_kreg.inp

In [ ]:
!cat traj.xyz

In [ ]:
import mlatom as ml

# Get initial condition
mol = ml.data.molecule.from_xyz_file('h2_md_kreg_init.xyz')
init_cond_db = ml.generate_initial_conditions(molecule = mol,
                                              generation_method = 'user-defined',
                                              file_with_initial_xyz_coordinates = 'h2_md_kreg_init.xyz',
                                              file_with_initial_xyz_velocities  = 'h2_md_kreg_init.vxyz')
init_mol = init_cond_db[0]

# Initialize model
model = ml.models.kreg(model_file='energies.unf', ml_program='MLatomF')
# model = ml.models.kreg(model_file='energies.unf', ml_program='MLatomF') # use KREG model from MLatomF
# model = ml.models.ani(model_file='energies_ani.pt')                     # or use an ANI model

# Run molecular dynamics
dyn = ml.md(model=model,
            molecule_with_initial_conditions = init_mol,
            ensemble='NVE',
            time_step=0.3,
            maximum_propagation_time = 30.0)

# Dump trajectory
traj = dyn.molecular_trajectory
traj.dump(filename='traj', format='plain_text')
traj.dump(filename='traj.h5', format='h5md')



# Spectroscopy

## Power spectra

In [ ]:
!cat ethanol_ps.inp

In [ ]:
!mlatom ethanol_ps.inp

In [ ]:
from IPython.display import Image
Image(filename='ps.png')

## IR spectra

In [ ]:
!cat ethanol_ir.inp

In [ ]:
!mlatom ethanol_ir.inp

In [ ]:
from IPython.display import Image
Image(filename='ir.png')

In [ ]:
import mlatom as ml

traj = ml.data.molecular_trajectory()
traj.load('ethanol_traj.h5',format='h5md')
dt = traj.steps[1].time - traj.steps[0].time
moldb = ml.data.molecular_database()
moldb.molecules = [each.molecule for each in traj.steps]

vib = ml.vibrational_spectrum(molecular_database=moldb, dt=dt)
vib.plot_infrared_spectrum(filename='ir_api.png')

from IPython.display import Image
Image(filename='ir_api.png')